In [1]:
def convert(fromunits,tounits,*argv):
    # handles the following (including prefixed versions)
    # g/l<-->mol/l (g/mol = required argv)
    # g<-->mol (g/mol)
    # X<-->X (note: if scalar entered for tounits the result is a dilution factor)
    # g or mol<-->l (g/mol)(mol/l) OR (g/l) ** detect first argv to decide

    def extract(units):
        # this function uses regular expressions (re) to extract the scalar, units, and magnitude factors from each input
        import re
        magnitudes={'f':10**-15,'p':10**-12,'n':10**-9,'u':10**-6,'m':10**-3,'c':10**-2,'d':0.1,'k':1000,'':1}
        r=re.compile('([0-9\.e\-]*)([fpnumcdk]?(?!ol)) ?(g|mol|l?)/?([fpnumcdk]?(?!ol)) ?(g|mol|l?)')
        m=r.findall(units)
        scalar = m[0][0]
        if scalar=='':
            scalar=1
        else:
            scalar = float(scalar)
        factor = magnitudes[m[0][1]]/magnitudes[m[0][3]]
        meastype = m[0][2]+'_'+m[0][4]
        #print(scalar,meastype,factor)
        return (scalar,meastype,factor)

    (f_s,f_m,f_f)=extract(fromunits)
    (t_s,t_m,t_f)=extract(tounits)
    
    conversion = f_m+'-->'+t_m
    factor = 0
    printstring = 'this was not set'
    
    if f_m==t_m:
        factor = (f_s*f_f/(t_s*t_f))
        if t_s==1:
            printstring = fromunits+' is @ '+tounits
        else:
            printstring = 'dilution factor is @'
    elif conversion in ['g_l-->mol_l','g_-->mol_']:
        (a_s,a_m,a_f)=extract(argv[0])
        if a_m=='g_mol':
            factor = (f_s*f_f)/(a_s*a_f)/(t_s*t_f)
            printstring = printstring = fromunits+' is @ '+tounits
        else:
            print('error: expected single argv entry Xg/mol but received '+a_m)
            
    elif conversion in ['mol_l-->g_l','mol_-->g_']:
        (a_s,a_m,a_f)=extract(argv[0])
        if a_m=='g_mol':
            factor = (f_s*f_f)*(a_s*a_f)/(t_s*t_f)
            printstring = printstring = fromunits+' is @ '+tounits
        else:
            print('error: expected single argv entry Xg/mol but received '+a_m)
    elif conversion=='g_-->l_':
        (a_s,a_m,a_f)=extract(argv[0])
        if a_m=='g_l':
            factor = (f_s*f_f)/(a_s*a_f)/(t_s*t_f)
            printstring = 'mix '+fromunits+' into @ '+tounits+' to make '+argv[0]
        else:
            (a2_s,a2_m,a2_f)=extract(argv[1])
            if a_m+'__'+a2_m=='g_mol__mol_l':
                factor = (f_s*f_f)/(a_s*a_f)/(a2_s*a2_f)/(t_s*t_f)
                printstring = 'mix '+fromunits+' into @ '+tounits+' to make '+argv[1]
            else:
                print('error, anticipated g/mol then mol/l argvs')
                
    elif conversion=='mol_-->l_':
        (a_s,a_m,a_f)=extract(argv[0])
        if a_m=='mol_l':
            factor = (f_s*f_f)/(a_s*a_f)/(t_s*t_f)
            printstring = 'mix '+fromunits+' into @ '+tounits+' to make '+argv[0]
        else:
            (a2_s,a2_m,a2_f)=extract(argv[1])
            if a_m+'__'+a2_m=='g_mol__g_l':
                factor = (f_s*f_f)*(a_s*a_f)/(a2_s*a2_f)/(t_s*t_f)
                printstring = 'mix '+fromunits+' into @ '+tounits+' to make '+argv[1]
            else:
                print('error, anticipated g/mol then g/l argvs')

    elif conversion=='l_-->g_':
        (a_s,a_m,a_f)=extract(argv[0])
        if a_m=='g_l':
            factor = (f_s*f_f)*(a_s*a_f)/(t_s*t_f)
            printstring = 'mix @ '+tounits+' into '+fromunits+' to make '+argv[0]
        else:
            (a2_s,a2_m,a2_f)=extract(argv[1])
            if a_m+'__'+a2_m=='g_mol__mol_l':
                factor = (f_s*f_f)*(a2_s*a2_f)*(a_s*a_f)/(t_s*t_f)
                printstring = 'mix @ '+tounits+' into '+fromunits+' to make '+argv[1]
            else:
                print('error, anticipated g/mol then mol/l argvs')
                
    elif conversion=='l_-->mol_':
        (a_s,a_m,a_f)=extract(argv[0])
        if a_m=='mol_l':
            factor = (f_s*f_f)*(a_s*a_f)/(t_s*t_f)
            printstring = 'mix @ '+tounits+' into '+fromunits+' to make '+argv[0]
        else:
            (a2_s,a2_m,a2_f)=extract(argv[1])
            if a_m+'__'+a2_m=='g_mol__g_l':
                factor = (f_s*f_f)*(a2_s*a2_f)/(a_s*a_f)/(t_s*t_f)
                printstring = 'mix @ '+tounits+' into '+fromunits+' to make '+argv[0]
            else:
                print('error, anticipated g/mol then g/l argvs')
                printstring = 'mix @ '+tounits+' into '+fromunits+' to make '+argv[1]

    else:
        print('not yet implemented')
    if any([factor > 1000,factor < 0.001]):
        factor_str = format(factor,'.2e')
    elif factor > 100:
        factor_str = format(factor,'.0f')
    elif factor > 10:
        factor_str = format(factor,'.1f')
    elif factor  > 1:
        factor_str = format(factor,'.2f')
    elif factor  > 0.1:
        factor_str = format(factor,'.3f')
    elif factor  > 0.01:
        factor_str = format(factor,'.4f')
    else:
        factor_str = format(factor,'.5f')

    


    print(printstring.replace('@',factor_str))
    return factor

In [7]:
convert('1200fmol/pl','mmol/ml')

1200fmol/pl is 1.20 mmol/ml


1.2

In [8]:
convert('5mmol','mg','500g/mol')

5mmol is 2.50e+03 mg


2500.0

In [9]:
convert('10mg/ml','nmol/ml','500g/mol')

10mg/ml is 2.00e+04 nmol/ml


20000.0

In [10]:
convert('5ml','mg','10g/l')

mix 50.0 mg into 5ml to make 10g/l


50.0

In [6]:
convert('2ng','ml','500g/mol','12mmol/l')

mix 2ng into 3.33e-07 ml to make 12mmol/l


3.333333333333333e-07